In [11]:
import os
import shutil
import pandas as pd
from Bio import SeqIO
import pandas as pd

# Define paths
fasta_file = "pdbbind_NL2020_rna.fasta"
source_dir = "pdbbind_dataset"
destination_dir = "pdbbind_dataset_rna"

# Define directories to search and copy
folders = [
    "parsed_rna3db_rnas_whole_pdb",
    "complex_database",
    "parsed_ligand_pdb",
    "parsed_ligand",
    "parsed_rna3db_rnas_whole"
]

# Read the FASTA file and extract IDs
def read_fasta_to_df(fasta_file):
    # Create a list to store sequence IDs and sequences
    records = []
    
    # Use SeqIO to parse the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        records.append([record.id, str(record.seq)])
    
    # Convert the list to a DataFrame
    df = pd.DataFrame(records, columns=['ID', 'Sequence'])
    return df

def remove_ids_from_df(df, ids_to_remove):
    # Filter the DataFrame to exclude rows with IDs in the ids_to_remove list
    filtered_df = df[~df['ID'].isin(ids_to_remove)]
    return filtered_df

# Function to search and copy matching files
def search_and_copy(fasta_ids, source_dir, destination_dir, folders):
    for folder in folders:
        source_folder = os.path.join(source_dir, folder)
        dest_folder = os.path.join(destination_dir, folder)

        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        for fasta_id in fasta_ids:
            # Search for files that start with the ID in the current folder
            for file_name in os.listdir(source_folder):
                if file_name.startswith(fasta_id):
                    source_file_path = os.path.join(source_folder, file_name)
                    dest_file_path = os.path.join(dest_folder, file_name)
                    
                    # Copy the file to the destination directory
                    shutil.copy2(source_file_path, dest_file_path)
                    print(f"Copied {file_name} to {dest_folder}")

# Main function to handle the process
def main():
    df = read_fasta_to_df(fasta_file)
    ids_to_remove = ["1q2t", "2kdq", "3sd3", "5uzz", "6d2u", "2rsk"]
    fasta_ids = remove_ids_from_df(df, ids_to_remove)['ID']
    search_and_copy(fasta_ids, source_dir, destination_dir, folders)


In [23]:
main()

Copied 5c7u_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 6e8s_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 5kpy_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 6c63_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 3slm_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 4nyg_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 2b57_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 4lvv_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 3mur_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 1f27_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 2xo1_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 2kgp_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 3fo6_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_whole_pdb
Copied 3jq4_rna.pdb to pdbbind_dataset_rna/parsed_rna3db_rnas_wh

In [32]:
import os
import subprocess

def convert_pdb_to_mol2(pdb_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Loop through all files in the pdb_dir
    for filename in os.listdir(pdb_dir):
        if filename.endswith(".pdb"):
            # Define the input and output file paths
            pdb_file = os.path.join(pdb_dir, filename)
            mol2_file = os.path.join(output_dir, filename.replace(".pdb", ".mol2"))
            
            # Use Open Babel to convert the PDB file to MOL2 format
            subprocess.run(['obabel', pdb_file, '-O', mol2_file])
            print(f"Converted {pdb_file} to {mol2_file}")
            
            # Modify the <TRIPOS>MOLECULE entry in the generated MOL2 file
            modify_mol2_molecule_name(mol2_file, filename)

def modify_mol2_molecule_name(mol2_file, pdb_filename):
    # Construct the new molecule name (e.g., "6hag_ligand")
    pdb_base = pdb_filename.split("_")[0]  # Extract the base name before the first underscore
    new_molecule_name = f"{pdb_base}_ligand"

    # Read the MOL2 file and modify the molecule name
    with open(mol2_file, 'r') as file:
        lines = file.readlines()

    # Find the <TRIPOS>MOLECULE section and replace the molecule name
    for i, line in enumerate(lines):
        if line.startswith('@<TRIPOS>MOLECULE'):
            lines[i + 1] = f"{new_molecule_name}\n"  # Replace the line after <TRIPOS>MOLECULE
            break

    # Write the modified content back to the MOL2 file
    with open(mol2_file, 'w') as file:
        file.writelines(lines)

# Define the directory containing PDB files and the output directory for MOL2 files
pdb_directory = "pdbbind_dataset_rna/parsed_ligand_pdb"
output_directory = "pdbbind_dataset_rna/parsed_ligand_mol2"

# Convert all PDB files to MOL2
convert_pdb_to_mol2(pdb_directory, output_directory)


1 molecule converted
1 molecule converted


Converted pdbbind_dataset_rna/parsed_ligand_pdb/5btp_AMZ_Q.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/5btp_AMZ_Q.mol2
Converted pdbbind_dataset_rna/parsed_ligand_pdb/4erj_ACA_B.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/4erj_ACA_B.mol2


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdbbind_dataset_rna/parsed_ligand_pdb/5v3f_74G_F.pdb)

1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdbbind_dataset_rna/parsed_ligand_pdb/6e8s_EKJ_G.pdb)

1 molecule converted


Converted pdbbind_dataset_rna/parsed_ligand_pdb/5v3f_74G_F.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/5v3f_74G_F.mol2
Converted pdbbind_dataset_rna/parsed_ligand_pdb/6e8s_EKJ_G.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/6e8s_EKJ_G.mol2


1 molecule converted
1 molecule converted


...

Converted pdbbind_dataset_rna/parsed_ligand_pdb/1yrj_AM2_E.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/1yrj_AM2_E.mol2
Converted pdbbind_dataset_rna/parsed_ligand_pdb/4lx6_29H_B.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/4lx6_29H_B.mol2


1 molecule converted
1 molecule converted


Converted pdbbind_dataset_rna/parsed_ligand_pdb/3la5_5AZ_L.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/3la5_5AZ_L.mol2
Converted pdbbind_dataset_rna/parsed_ligand_pdb/2b57_6AP_L.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/2b57_6AP_L.mol2


1 molecule converted
1 molecule converted


Converted pdbbind_dataset_rna/parsed_ligand_pdb/4lvv_FFO_B.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/4lvv_FFO_B.mol2
Converted pdbbind_dataset_rna/parsed_ligand_pdb/4qln_2BA_B.pdb to pdbbind_dataset_rna/parsed_ligand_mol2_/4qln_2BA_B.mol2


In [12]:
import os
import shutil
import pandas as pd

source_dir = "pdbbind_dataset_rna"
destination_dir = "pdbbind_rna_set_"

folders = [
    "parsed_rna3db_rnas_whole_pdb",
    "parsed_ligand_pdb",
    # "parsed_ligand_mol2"
]

df = read_fasta_to_df(fasta_file)
ids_to_remove = ["1q2t", "2kdq", "3sd3", "5uzz", "6d2u", "2rsk"]
fasta_ids = remove_ids_from_df(df, ids_to_remove)['ID']


for fasta_id in fasta_ids:
    output_folder = os.path.join(destination_dir, fasta_id)
    os.makedirs(output_folder, exist_ok=True)
    for folder in folders:
        source_folder = os.path.join(source_dir, folder)
        for file_name in os.listdir(source_folder):
                if file_name.startswith(fasta_id):
                    source_file_path = os.path.join(source_folder, file_name)
                    dest_file_path = os.path.join(output_folder, file_name)
                    shutil.copy2(source_file_path, dest_file_path)
                    break
    

In [30]:
df =pd.read_csv("pdbbind_NL_labels.csv")

In [31]:
df[df['pdb'].isin(fasta_ids)].to_csv("pdbbind_NL_labels_rna.csv", index=False)

In [34]:
len(fasta_ids)

118

In [13]:
import os
import fnmatch
import subprocess
pdbbind_rna_set_dir = "pdbbind_rna_set_"
def convert_pdb_to_mol2(pdbbind_rna_set_dir):
    for pdbid in os.listdir(pdbbind_rna_set_dir):
        current_dir = os.path.join(pdbbind_rna_set_dir, pdbid)
        for filename in os.listdir(current_dir):
            pattern = f"{pdbid}_*_*.pdb"
            if fnmatch.fnmatch(filename, pattern):
                pdb_file = os.path.join(current_dir, filename)
                mol2_file = os.path.join(current_dir, filename.replace(".pdb", ".mol2"))
                # Use Open Babel to convert the PDB file to MOL2 format
                subprocess.run(['obabel', pdb_file, '-O', mol2_file])
                print(f"Converted {pdb_file} to {mol2_file}")

    

In [14]:
convert_pdb_to_mol2(pdbbind_rna_set_dir)

1 molecule converted
1 molecule converted


Converted pdbbind_rna_set_/3mur/3mur_C2E_C.pdb to pdbbind_rna_set_/3mur/3mur_C2E_C.mol2
Converted pdbbind_rna_set_/4lvx/4lvx_H4B_C.pdb to pdbbind_rna_set_/4lvx/4lvx_H4B_C.mol2


1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdbbind_rna_set_/6e8u/6e8u_HZD_G.pdb)

1 molecule converted


Converted pdbbind_rna_set_/2o3w/2o3w_PAR_C.pdb to pdbbind_rna_set_/2o3w/2o3w_PAR_C.mol2
Converted pdbbind_rna_set_/6e8u/6e8u_HZD_G.pdb to pdbbind_rna_set_/6e8u/6e8u_HZD_G.mol2


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdbbind_rna_set_/6hmo/6hmo_GDZ_C.pdb)

1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdbbind_rna_set_/6c63/6c63_EKJ_H.pdb)

1 molecule converted


...

1 molecule converted
1 molecule converted


Converted pdbbind_rna_set_/3ger/3ger_6GU_B.pdb to pdbbind_rna_set_/3ger/3ger_6GU_B.mol2
Converted pdbbind_rna_set_/3b31/3b31_IRI_C.pdb to pdbbind_rna_set_/3b31/3b31_IRI_C.mol2


1 molecule converted
1 molecule converted


Converted pdbbind_rna_set_/3fo4/3fo4_6GU_B.pdb to pdbbind_rna_set_/3fo4/3fo4_6GU_B.mol2
Converted pdbbind_rna_set_/5lwj/5lwj_GTP_B.pdb to pdbbind_rna_set_/5lwj/5lwj_GTP_B.mol2
Converted pdbbind_rna_set_/2mxs/2mxs_PAR_B.pdb to pdbbind_rna_set_/2mxs/2mxs_PAR_B.mol2


1 molecule converted
